In [8]:
import socket
import matplotlib.pyplot as plt
import time
import numpy as np
import sounddevice as sd
from scipy.signal import blackman
import csv

In [9]:
import csv
from datetime import datetime

def save_waveform_data(timestamps, data, base_file_name="waveform_data"):
    """Save the waveform data to a CSV file with a name based on the current date and time.

    Parameters:
    - timestamps: A list of timestamp values.
    - data: A list of amplifier data values.
    - base_file_name: The base name of the file to save the data to, without extension.
    """
    # Get the current date and time, formatted as 'YYYY-MM-DD_HH-MM-SS'
    current_time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    # Create the file name by appending the current time to the base file name
    file_name = f"{base_file_name}_{current_time_str}.csv"
    
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Timestamp (s)', 'Voltage (uV)'])  # Write header
        for timestamp, voltage in zip(timestamps, data):
            writer.writerow([timestamp, voltage])
    
    print(f'Data successfully saved to {file_name}')

# Example usage
# Assuming amplifierTimestamps and amplifierData contain your data
# save_waveform_data(amplifierTimestamps, amplifierData)



In [10]:
import numpy as np
import sounddevice as sd
from numpy.random import normal
from datetime import datetime
import csv

Fs = 44100  # Sampling frequency
DURATION = 1  # Duration of the sound in seconds

def generate_sound(sel):
    """Generate a sound based on selection."""
    t = np.linspace(0, DURATION, int(Fs * DURATION), endpoint=False)
    if sel == 1:  # Blackman filtered sine
        frequency = 440  # Example frequency
        sound = np.sin(2 * np.pi * frequency * t) * np.blackman(len(t))
    elif sel == 2:  # Blackman filtered Gaussian
        sound = normal(0, 1, len(t)) * np.blackman(len(t))
    elif sel == 3:  # Plain Gaussian
        sound = normal(0, 1, len(t))
    elif sel == 4:  # Homophasic-Antiphasic
        sound = np.sin(2 * np.pi * 440 * t) - np.sin(2 * np.pi * 440 * t + np.pi)
    else:
        raise ValueError("Selection must be between 1 and 4.")
    return sound

import random

def play_sound_sel(sel, sound_events, elapsed_time, Fs=44100):
    """Play the selected sound."""
    channel = random.choice([1, 2, 3])  # 1=left, 2=right, 3=silence
    sound_mono = generate_sound(sel)
    if channel == 1:  # Left
        sound_stereo = np.column_stack((sound_mono, np.zeros(len(sound_mono))))
    elif channel == 2:  # Right
        sound_stereo = np.column_stack((np.zeros(len(sound_mono)), sound_mono))
    else:  # Silence
        sound_stereo = np.column_stack((np.zeros(len(sound_mono)), np.zeros(len(sound_mono))))
    
    sound_events.append({'time': elapsed_time, 'sound': sel, 'channel': channel})
    sd.play(sound_stereo, Fs)
    sd.wait()

def save_sound_events(sound_events, file_name='sound_events.csv'):
    """Save sound events to a CSV file."""
    current_time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"sound_{current_time_str}.csv"
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time (s)', 'Sound Selection', 'Channel'])
        for event in sound_events:
            writer.writerow([event['time'], event['sound'], event['channel']])
    print(f'Sound Data successfully saved to {file_name}')



In [11]:
# Constants related to the data format and TCP streaming protocol
FRAMES_PER_BLOCK = 128  # Number of frames per block of data
SAMPLE_RATE = 30000  # This should be set to the actual sample rate obtained from the RHX software
TIMESTAMP_BYTES = 4  # Number of bytes for each timestamp in the data stream
SAMPLE_BYTES = 2  # Number of bytes for each data sample in the data stream
MAGIC_NUMBER = 0x2ef07a08  # Expected magic number indicating the start of a new data block
WAVEFORM_BYTES_PER_FRAME = TIMESTAMP_BYTES + SAMPLE_BYTES  # Total bytes per frame, adjust if more channels
SIZE_OF_MAGIC_NUMBER = 4  # Byte size of the magic number
COMMAND_BUFFER_SIZE = 1024  # 1024 bytes or 1 KB
WAVEFORM_BUFFER_SIZE = 800000 


In [12]:
#! /bin/env python3
# Adrian Foy September 2023

"""Example demonstrating reading of 1 second waveform data (wideband amplifier
data on channel A-010) using TCP command socket to control RHX software and TCP
waveform socket to read amplifier data.

In order to run this example script successfully, the Intan RHX software
should first be started, and through Network -> Remote TCP Control.

Command Output should open a connection at 127.0.0.1, Port 5000.
Status should read "Pending".

Waveform Output (in the Data Output tab) should open a connection at 127.0.0.1,
Port 5001. Status should read "Pending" for the Waveform Port (Spike Port is
unused for this example, and can be left disconnected).

Once these ports are opened, this script can be run to acquire ~1 second of
wideband data from channel A-010, which can then be plotted assuming
"matplotlib" is installed.
"""

import time
import socket
import numpy as np
# In order to plot the data, 'matplotlib' is required.
# If plotting is not needed, calls to plt can be removed and the data
# will still be present within the ReadWaveformDataDemo() function.
# 'matplotlib' can be installed with the command 'pip install matplotlib'
import matplotlib.pyplot as plt



# Exception classes for specific error scenarios
class GetSampleRateFailure(Exception):
    pass

class InvalidMagicNumber(Exception):
    pass

# Utility functions to read data from byte arrays
def readUint32(data, index):
    result = int.from_bytes(data[index:index+4], byteorder='little', signed=False)
    return result, index + 4

def readInt32(data, index):
    result = int.from_bytes(data[index:index+4], byteorder='little', signed=True)
    return result, index + 4

def readUint16(data, index):
    result = int.from_bytes(data[index:index+2], byteorder='little', signed=False)
    return result, index + 2


In [13]:

        
def init_tcp_connections():
    """Initialize TCP connections for command and waveform data."""
    scommand = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    scommand.connect(('127.0.0.1', 5000))
    print('Connected to TCP command server.')

    swaveform = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    swaveform.connect(('127.0.0.1', 5001))
    print('Connected to TCP waveform server.')

    return scommand, swaveform

def prepare_for_data_collection(scommand):
    """Prepare Intan RHX software for data collection by 
    stopping the controller if running, querying the sample rate, and clearing data outputs."""
    # Stop the controller if it's running
    scommand.sendall(b'get runmode')
    runmode = scommand.recv(COMMAND_BUFFER_SIZE).decode()
    if "RunMode Run" in runmode:
        scommand.sendall(b'set runmode stop')
        time.sleep(1)  # Wait for the command to take effect
    
    # Query the sample rate
    scommand.sendall(b'get sampleratehertz')
    sample_rate_response = scommand.recv(COMMAND_BUFFER_SIZE).decode()
    print(f"Sample Rate Response: {sample_rate_response}")
    
    # Clear all data outputs to ensure a clean start
    scommand.sendall(b'execute clearalldataoutputs')
    time.sleep(0.1)  # Wait for the command to take effect


def start_data_collection_with_sound(scommand, swaveform, sound_selection, repeats, interval, start_delay, recording_dur):
    sound_events = []
    start_time = time.time()

    # Start recording
    scommand.sendall(b'set runmode run')
    time.sleep(start_delay)  # Initial delay before starting sound playback
    
    elapsed_time = time.time() - start_time
    sound_played = 0  # Counter for how many sounds have been played

    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time >= recording_dur:
            break  # Exit loop if the recording duration is reached or exceeded
        
        # Check if we can play another sound without exceeding recording duration
        if sound_played < repeats and elapsed_time + interval <= recording_dur:
            play_sound_sel(sound_selection, sound_events, elapsed_time)
            sound_played += 1
            next_sound_time = min(interval, recording_dur - elapsed_time)
            time.sleep(next_sound_time)  # Sleep until it's time for the next sound or until recording duration is reached


    # Stop recording
    scommand.sendall(b'set runmode stop')
    
    # There should be a brief delay here to ensure the stop command is processed
    time.sleep(1)

    # Fetch and return rawData
    rawData = swaveform.recv(WAVEFORM_BUFFER_SIZE)
    return sound_events, rawData

    


def save_waveform_and_sound_events(sound_events, rawData):
    """Save collected waveform data and sound events."""
    # Assuming the functions to process rawData into amplifierTimestamps and amplifierData are defined
    amplifierTimestamps, amplifierData = process_raw_waveform_data(rawData)
    
    # Save waveform data
    save_waveform_data(amplifierTimestamps, amplifierData, "waveform_data")
    
    # Save sound events
    save_sound_events(sound_events, "sound_events.csv")

def process_raw_waveform_data(rawData, waveformBytesPerBlock, FRAMES_PER_BLOCK, timestep):
    """Process raw waveform data into timestamps and data points."""
    amplifierTimestamps = []
    amplifierData = []
    rawIndex = 0
    numCompleteBlocks = len(rawData) // waveformBytesPerBlock

    for _ in range(numCompleteBlocks):
        # Ensure the block is complete with enough data
        if rawIndex + waveformBytesPerBlock <= len(rawData):
            # Read and verify the magic number for each block
            magicNumber, rawIndex = readUint32(rawData, rawIndex)
            if magicNumber != 0x2ef07a08:
                raise InvalidMagicNumber('Error... magic number incorrect')

            # Process each frame in the block
            for _ in range(FRAMES_PER_BLOCK):
                # Read the timestamp for each frame
                rawTimestamp, rawIndex = readInt32(rawData, rawIndex)
                # Convert rawTimestamp according to the timestep to get the actual timestamp
                amplifierTimestamps.append(rawTimestamp * timestep)

                # Read the sample data for each frame and convert to microvolts
                rawSample, rawIndex = readUint16(rawData, rawIndex)
                # Convert rawSample to microvolts using the given formula
                amplifierData.append(0.195 * (rawSample - 32768))
        else:
            # Handle incomplete block or data format mismatch
            print("Incomplete block received at the end or data format mismatch.")
            break

    return amplifierTimestamps, amplifierData




In [14]:
def ReadWaveformDataDemo(sound_selection, repeats, interval, recording_dur, start_delay=3):
    """Function to demonstrate reading waveform data and playing sounds."""
    try:
        # Initialize TCP connections
        scommand, swaveform = init_tcp_connections()

        # Stop controller if running, query sample rate, and prepare for data collection
        prepare_for_data_collection(scommand)

        # Start data collection and sound playback
        sound_events = start_data_collection_with_sound(scommand, swaveform, sound_selection, repeats, interval, start_delay, recording_dur)

        # Save waveform and sound event data
        save_waveform_and_sound_events(sound_events)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        # Close TCP connections
        scommand.close()
        swaveform.close()
        
        
# Main execution block
if __name__ == '__main__':
    sound_selection = 4  # Example sound selection
    repeats = 3  # Number of times to play the sound
    interval = 2  # Interval between sounds, in seconds
    recording_dur = 10  # Total duration of recording
    ReadWaveformDataDemo(sound_selection, repeats, interval, recording_dur, start_delay=3)

Connected to TCP command server.
Connected to TCP waveform server.
Sample Rate Response: Return: SampleRateHertz 10000
